## Single output tuning for molecules


In [1]:
%reload_ext autoreload
%autoreload 2

import time

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pycm import ConfusionMatrix
import wandb
import pandas as pd
import numpy as np 

from gpt3forchem.molecules.data import get_data
from gpt3forchem.molecules.baseline.classification import LogisticClassifierBaseline
from gpt3forchem.molecules.constants import TARGETS, FEATURES, CAT_TARGETS, TEXT
from gpt3forchem.molecules.create_prompts import create_single_property_forward_prompts
from gpt3forchem.fine_tune import fine_tune
from gpt3forchem.query_model import query_gpt3, extract_prediction

In [2]:
df = get_data()


In [3]:
def run_baseline(train_df, test_df):
    train_size = len(train_df)
    test_size = len(test_df)
    run = wandb.init(
        project="gpt-3",
        job_type="train-baseline-model",
        config={
            "train_size": train_size,
            "test_size": test_size,
            "features": FEATURES,
            "targets": [CAT_TARGETS[0]],
            "ds": "molecule",
        },
    )
    baseline = LogisticClassifierBaseline(43)
    baseline.fit(train_df[FEATURES], train_df[CAT_TARGETS[0]])
    predictions = baseline.predict(test_df[FEATURES].values)

    run.log(
        {
            "distribution confusion matrix": wandb.plot.confusion_matrix(
                y_true=test_df[CAT_TARGETS[0]].values,
                preds=predictions,
                class_names=np.arange(5),
            )
        }
    )

    cm = ConfusionMatrix(
        test_df[CAT_TARGETS[0]].values,
        predictions,
    )

    run.log(
        {"ACC": cm.ACC, "ACC_macro": cm.ACC_Macro, "F1_micro": cm.F1_Micro, "F1_macro": cm.F1_Macro}
    )

    print(
        {"ACC": cm.ACC, "ACC_macro": cm.ACC_Macro, "F1_micro": cm.F1_Micro, "F1_macro": cm.F1_Macro}
    )

    run.finish()


In [4]:
def create_prompt_fine_tune(train_df, test_df, text=0):
    print(f"Creating fine tune prompts for {TEXT[text]}")
    train_prompts = create_single_property_forward_prompts(train_df, CAT_TARGETS[0], TEXT[text])
    valid_prompts = create_single_property_forward_prompts(test_df, CAT_TARGETS[0], TEXT[text])

    filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
    train_filename = f"run_files/{filename_base}_train_prompts_molecule_{train_size}.jsonl"
    valid_filename = f"run_files/{filename_base}_valid_prompts_molecule_{test_size}.jsonl"
    # to save money, just run a small valid frame
    valid_small_filename = (
        f"run_files/{filename_base}_validsmall_prompts_molecule_{test_size}.jsonl"
    )
    train_prompts.to_json(train_filename, orient="records", lines=True)
    valid_prompts.to_json(valid_filename, orient="records", lines=True)
    valid_prompts.sample(100).to_json(valid_small_filename, orient="records", lines=True)

    fine_tune(train_filename, valid_small_filename)
    return train_prompts, valid_prompts

In [5]:
def test_gpt3(model_id, finetune_id, valid_prompts, test_df, max_samples: int = 100):
    completions = query_gpt3(model_id, valid_prompts.iloc[:max_samples])
    predictions = [int(extract_prediction(completion)) for completion in completions]
    true = test_df.iloc[:max_samples][CAT_TARGETS[0]].values

    run = wandb.init(
        project="gpt-3",
        job_type="test-finetuned-model",
        config={
            "train_size": train_size,
            "test_size": test_size,
            "features": FEATURES,
            "targets": [CAT_TARGETS[0]],
            "ds": "molecule",
            "modelname": model_id,
            "finetune_run_id": finetune_id,
        },
    )

    run.log(
        {
            "distribution confusion matrix": wandb.plot.confusion_matrix(
                y_true=true,
                preds=predictions,
            )
        }
    )

    cm = ConfusionMatrix(
        true,
        predictions,
    )

    run.log(
        {"ACC": cm.ACC, "ACC_macro": cm.ACC_Macro, "F1_micro": cm.F1_Micro, "F1_macro": cm.F1_Macro}
    )

    print(
        {"ACC": cm.ACC, "ACC_macro": cm.ACC_Macro, "F1_micro": cm.F1_Micro, "F1_macro": cm.F1_Macro}
    )
    run.finish()


## Small data limit ($N = 100$)


In [6]:
train_df, test_df = train_test_split(df, train_size=100, stratify=df[CAT_TARGETS[0]])
train_size = len(train_df)
test_size = len(test_df)
print(f"{len(train_df)} train points and {len(test_df)} test points")


100 train points and 18409 test points


### Compute the baseline


In [44]:
run_baseline(train_df, test_df)


/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

{'ACC': {0: 0.8861969688739204, 1: 0.8027051985441903, 2: 0.7447444184909555, 3: 0.761855614101798, 4: 0.8527893964908468}, 'ACC_macro': 0.8096583193003422, 'F1_micro': 0.5241457982508556, 'F1_macro': 0.5179678782595067}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.80966
F1_macro,0.51797
F1_micro,0.52415


#### GPT3 on SMILES

In [47]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=0)


Creating fine tune prompts for smiles


Upload progress: 100%|██████████| 9.91k/9.91k [00:00<00:00, 4.50Mit/s]


Uploaded file from run_files/2022-07-03-14-39-19_train_prompts_molecule_100.jsonl: file-PrmjBFTjZeEvYABIV9x22Fbv


Upload progress: 100%|██████████| 11.1k/11.1k [00:00<00:00, 14.5Mit/s]


Uploaded file from run_files/2022-07-03-14-39-19_validsmall_prompts_molecule_18409.jsonl: file-A818SFA8AyhMkTXi445mRFdI
Created fine-tune: ft-qHqcHsf69wW24yFoXKR1CqyZ
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 14:39:26] Created fine-tune: ft-qHqcHsf69wW24yFoXKR1CqyZ
[2022-07-03 14:39:32] Fine-tune costs $0.01
[2022-07-03 14:39:33] Fine-tune enqueued. Queue number: 0
[2022-07-03 14:39:37] Fine-tune started
[2022-07-03 14:40:08] Completed epoch 1/4
[2022-07-03 14:40:23] Completed epoch 2/4
[2022-07-03 14:40:38] Completed epoch 3/4
[2022-07-03 14:40:53] Completed epoch 4/4
[2022-07-03 14:41:16] Uploaded model: ada:ft-epfl-2022-07-03-12-41-14
[2022-07-03 14:41:19] Uploaded result file: file-K9yKddDBFuiwGVJREkCEcm8A
[2022-07-03 14:41:20] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-12-41-14 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220703_144130-ft-qHqcHsf69wW24yFoXKR1CqyZ
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-qHqcHsf69wW24yFoXKR1CqyZ
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-qHqcHsf69wW24yFoXKR1CqyZ
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                


🎉 wandb sync completed successfully


wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
wandb:                training_loss █▅▃▄▄▂▂▃▄▃▄▂▂▄▂▃▃▂▂▂▁▃▄▂▂▁▂▂▂▂▃▁▂▂▁▂▁▁▁▂
wandb:   training_sequence_accuracy ▁▁█▁██▁▁▁▁▁▁█▁█████▁█▁▁▁███████▁████████
wandb:      training_token_accuracy ▁▆█▆██▆▆▆▆▆▆█▆█████▆█▆▆▆███████▆████████
wandb:              validation_loss █▄▃▄▄▄▂▂▃▃▄▃▂▂▃▃▄▂▂▂▃▂▂▂▂▃▂▃▂▂▃▃▂▁▃▂▂▃▃
wandb: validation_sequence_accuracy ▁█▁▁▁▁▁█▁▁▁▁█▁▁█▁▁█▁███▁▁▁▁▁▁█▁▁▁█▁▁██▁
wandb:    validation_token_accuracy ▁█▆▆▆▆▆█▆▆▆▆█▆▆█▆▆█▆███▆▆▆▆▆▆█▆▆▆█▆▆██▆
wandb: 
wandb: Run summary:
wandb:             elapsed_examples 402.0
wandb:               elapsed_tokens 18842.0
wandb:             fine_tuned_model ada:ft-epfl-2022-07-...
wandb:                       status succeeded
wandb:                training_loss 0.17082
wandb:   training_sequence_accuracy 1.0
wandb:      training_token_accuracy 1.0
wandb:         

In [48]:
test_gpt3("ada:ft-epfl-2022-07-03-12-41-14", "ft-qHqcHsf69wW24yFoXKR1CqyZ", valid_prompts, test_df)

{'ACC': {0: 0.88, 1: 0.79, 2: 0.59, 3: 0.75, 4: 0.79}, 'ACC_macro': 0.76, 'F1_micro': 0.4, 'F1_macro': 0.41704646676486484}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.76
F1_macro,0.41705
F1_micro,0.4


#### GPT3 on SELFIES

In [49]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=1)


Creating fine tune prompts for selfies


Upload progress: 100%|██████████| 21.9k/21.9k [00:00<00:00, 13.2Mit/s]


Uploaded file from run_files/2022-07-03-14-52-24_train_prompts_molecule_100.jsonl: file-7WuTIHOhMIhmtS3NQd4yVYA6


Upload progress: 100%|██████████| 22.6k/22.6k [00:00<00:00, 33.4Mit/s]


Uploaded file from run_files/2022-07-03-14-52-24_validsmall_prompts_molecule_18409.jsonl: file-oZIcVqmi5NPNU4gERt4EKbHI
Created fine-tune: ft-KSOPxV6MYzu5chJGYdMDfV2j
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 14:52:32] Created fine-tune: ft-KSOPxV6MYzu5chJGYdMDfV2j
[2022-07-03 14:52:45] Fine-tune costs $0.02
[2022-07-03 14:52:45] Fine-tune enqueued. Queue number: 0
[2022-07-03 14:52:48] Fine-tune started
[2022-07-03 14:53:19] Completed epoch 1/4
[2022-07-03 14:53:35] Completed epoch 2/4
[2022-07-03 14:53:50] Completed epoch 3/4
[2022-07-03 14:54:05] Completed epoch 4/4
[2022-07-03 14:54:26] Uploaded model: ada:ft-epfl-2022-07-03-12-54-24
[2022-07-03 14:54:29] Uploaded result file: file-Q9oJJijPyGXbkokfGNAd3k6o
[2022-07-03 14:54:30] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-12-54-24 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220703_145441-ft-KSOPxV6MYzu5chJGYdMDfV2j
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-KSOPxV6MYzu5chJGYdMDfV2j
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-KSOPxV6MYzu5chJGYdMDfV2j
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                


🎉 wandb sync completed successfully


wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:                training_loss █▃▃▃▄▂▂▂▃▄▂▂▂▃▂▂▃▂▃▁▂▃▂▁▂▂▁▂▂▂▂▁▁▁▂▂▂▂▂▁
wandb:   training_sequence_accuracy ▁▁█▁█▁▁▁▁▁▁███▁▁▁▁▁██▁█▁█▁█▁▁█▁█████▁███
wandb:      training_token_accuracy ▁▅█▅█▅▅▅▅▅▅███▅▅▅▅▅██▅█▅█▅█▅▅█▅█████▅███
wandb:              validation_loss █▄▃▃▄▃▅▃▃▃▃▂▄▃▂▂▂▂▂▃▃▂▂▃▂▂▂▂▂▂▂▂▃▁▂▃▃▂
wandb: validation_sequence_accuracy ▁▁▁▁█▁█▁▁██▁▁▁▁█▁███▁█▁▁██▁▁█▁▁▁▁██▁▁█
wandb:    validation_token_accuracy ▁▆▆▆█▆█▆▆██▆▆▆▆█▆███▆█▆▆██▆▆█▆▆▆▆██▆▆█
wandb: 
wandb: Run summary:
wandb:             elapsed_examples 401.0
wandb:               elapsed_tokens 42801.0
wandb:             fine_tuned_model ada:ft-epfl-2022-07-...
wandb:                       status succeeded
wandb:                training_loss 0.04701
wandb:   training_sequence_accuracy 1.0
wandb:      training_token_accuracy 1.0
wandb:            

In [52]:
test_gpt3("ada:ft-epfl-2022-07-03-12-54-24", "ft-KSOPxV6MYzu5chJGYdMDfV2j", valid_prompts, test_df)

{'ACC': {0: 0.73, 1: 0.7, 2: 0.73, 3: 0.74, 4: 0.56}, 'ACC_macro': 0.6920000000000001, 'F1_micro': 0.23, 'F1_macro': 0.217015093015093}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.692
F1_macro,0.21702
F1_micro,0.23


#### GPT-3 on names

In [7]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=2)

Creating fine tune prompts for iupac_name


Upload progress: 100%|██████████| 12.9k/12.9k [00:00<00:00, 6.40Mit/s]


Uploaded file from run_files/2022-07-04-06-37-58_train_prompts_molecule_100.jsonl: file-oa3j7kpChV2POBalhMHfiJ40


Upload progress: 100%|██████████| 13.7k/13.7k [00:00<00:00, 20.1Mit/s]


Uploaded file from run_files/2022-07-04-06-37-58_validsmall_prompts_molecule_18409.jsonl: file-Q6E4sTvlks4JFWAQV21pJqCz
Created fine-tune: ft-9Xa22xxtYKI1sF7gs3DVdmxC
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-04 06:38:07] Created fine-tune: ft-9Xa22xxtYKI1sF7gs3DVdmxC
[2022-07-04 06:38:13] Fine-tune costs $0.01
[2022-07-04 06:38:14] Fine-tune enqueued. Queue number: 0
[2022-07-04 06:38:17] Fine-tune started
[2022-07-04 06:38:49] Completed epoch 1/4
[2022-07-04 06:39:04] Completed epoch 2/4
[2022-07-04 06:39:19] Completed epoch 3/4
[2022-07-04 06:39:34] Completed epoch 4/4
[2022-07-04 06:39:56] Uploaded model: ada:ft-epfl-2022-07-04-04-39-54
[2022-07-04 06:39:59] Uploaded result file: file-AHWlG7oBPPfWWMK0mDCCWGS1
[2022-07-04 06:40:00] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-04-04-39-54 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220704_064014-ft-9Xa22xxtYKI1sF7gs3DVdmxC
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-9Xa22xxtYKI1sF7gs3DVdmxC
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-9Xa22xxtYKI1sF7gs3DVdmxC
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                


🎉 wandb sync completed successfully


wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:                training_loss ▇▅█▄▄▄▄▃▄▄▄▃▃▂▅▃▃▂▃▃▃▂▂▂▂▂▃▃▃▃▂▁▂▁▂▂▂▁▁▁
wandb:   training_sequence_accuracy ▁▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁▁▁▁▁█▁▁██▁█▁▁██▁██▁████
wandb:      training_token_accuracy ▁▅▅▅▅▅▅▅▅▅█▅▅█▅▅▅▅▅▅▅█▅▅██▅█▅▅██▅██▅████
wandb:              validation_loss █▃▃▆▂▃▂▂▃▅▃▃▂▂▂▅▂▂▂▂▃▁▂▂▄▆▄▃▁▃▂▁▂▂▃▂▃▃
wandb: validation_sequence_accuracy ▁▁▁▁▁▁▁█▁▁▁▁███▁▁▁███▁▁▁▁▁▁▁█▁▁██▁▁▁▁█
wandb:    validation_token_accuracy ▁▆▆▆▆▆▆█▆▆▆▆███▆▆▆███▆▆▆▆▆▆▆█▆▆██▆▆▆▆█
wandb: 
wandb: Run summary:
wandb:             elapsed_examples 401.0
wandb:               elapsed_tokens 21665.0
wandb:             fine_tuned_model ada:ft-epfl-2022-07-...
wandb:                       status succeeded
wandb:                training_loss 0.11967
wandb:   training_sequence_accuracy 0.0
wandb:      training_token_accuracy 0.66667
wandb:        

In [8]:
test_gpt3("ada:ft-epfl-2022-07-04-04-39-54", "ft-9Xa22xxtYKI1sF7gs3DVdmxC", valid_prompts, test_df)

wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin


{'ACC': {0: 0.82, 1: 0.7, 2: 0.75, 3: 0.69, 4: 0.84}, 'ACC_macro': 0.76, 'F1_micro': 0.4, 'F1_macro': 0.39273675934735647}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.76
F1_macro,0.39274
F1_micro,0.4


## Medium size dataset ($N=500$)

In [6]:
train_df, test_df = train_test_split(df, train_size=500, stratify=df[CAT_TARGETS[0]])
train_size = len(train_df)
test_size = len(test_df)
print(f"{len(train_df)} train points and {len(test_df)} test points")


500 train points and 18009 test points


### Train a baseline

In [7]:
run_baseline(train_df, test_df)


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin


/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

{'ACC': {0: 0.9411405408406908, 1: 0.8687323005164085, 2: 0.8462990726858792, 3: 0.855072463768116, 4: 0.9262590926758842}, 'ACC_macro': 0.8875006940973957, 'F1_micro': 0.7187517352434893, 'F1_macro': 0.7198819297229052}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.8875
F1_macro,0.71988
F1_micro,0.71875


### GPT-3 on SMILES

In [8]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=0)


Creating fine tune prompts for smiles


Upload progress: 100%|██████████| 52.5k/52.5k [00:00<00:00, 27.9Mit/s]


Uploaded file from run_files/2022-07-03-15-40-58_train_prompts_molecule_500.jsonl: file-UpKzwSTMZ92uKyqAeCys2qjY


Upload progress: 100%|██████████| 10.4k/10.4k [00:00<00:00, 19.5Mit/s]


Uploaded file from run_files/2022-07-03-15-40-58_validsmall_prompts_molecule_18009.jsonl: file-lEex9MuVk41n2pKhPakEE3w2
Created fine-tune: ft-SYoz2NDeVkcnlrr4x8qrLNzD
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 15:41:07] Created fine-tune: ft-SYoz2NDeVkcnlrr4x8qrLNzD
[2022-07-03 15:41:12] Fine-tune costs $0.04
[2022-07-03 15:41:12] Fine-tune enqueued. Queue number: 0
[2022-07-03 15:41:15] Fine-tune started
[2022-07-03 15:42:42] Completed epoch 1/4
[2022-07-03 15:43:54] Completed epoch 2/4
[2022-07-03 15:45:07] Completed epoch 3/4
[2022-07-03 15:46:20] Completed epoch 4/4
[2022-07-03 15:46:45] Uploaded model: ada:ft-epfl-2022-07-03-13-46-43
[2022-07-03 15:46:48] Uploaded result file: file-2SZYAFeSu9lkBsJMbgqIN9jZ
[2022-07-03 15:46:48] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-13-46-43 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220703_154700-ft-SYoz2NDeVkcnlrr4x8qrLNzD
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-SYoz2NDeVkcnlrr4x8qrLNzD
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-SYoz2NDeVkcnlrr4x8qrLNzD
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
wandb:                tr

🎉 wandb sync completed successfully


In [9]:
test_gpt3("ada:ft-epfl-2022-07-03-13-46-43", "ft-SYoz2NDeVkcnlrr4x8qrLNzD", valid_prompts, test_df)

{'ACC': {0: 0.9, 1: 0.87, 2: 0.84, 3: 0.87, 4: 0.92}, 'ACC_macro': 0.8800000000000001, 'F1_micro': 0.7, 'F1_macro': 0.6967912860595787}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.88
F1_macro,0.69679
F1_micro,0.7


### GPT-3 on SELFIES

In [10]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=1)


Creating fine tune prompts for selfies


Upload progress: 100%|██████████| 120k/120k [00:00<00:00, 72.5Mit/s]


Uploaded file from run_files/2022-07-03-15-57-57_train_prompts_molecule_500.jsonl: file-O8fl0nF9ob6gYN1wxNnZ7d4y


Upload progress: 100%|██████████| 23.6k/23.6k [00:00<00:00, 46.0Mit/s]


Uploaded file from run_files/2022-07-03-15-57-57_validsmall_prompts_molecule_18009.jsonl: file-odcg4f4IW0be3IiDu31YBdJe
Created fine-tune: ft-pmx4lEQJw1L3rPR1PlHUzGXA
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 15:58:07] Created fine-tune: ft-pmx4lEQJw1L3rPR1PlHUzGXA
[2022-07-03 15:58:17] Fine-tune costs $0.09
[2022-07-03 15:58:17] Fine-tune enqueued. Queue number: 0
[2022-07-03 15:58:19] Fine-tune started
[2022-07-03 15:59:45] Completed epoch 1/4
[2022-07-03 16:00:57] Completed epoch 2/4
[2022-07-03 16:02:08] Completed epoch 3/4
[2022-07-03 16:03:19] Completed epoch 4/4
[2022-07-03 16:03:39] Uploaded model: ada:ft-epfl-2022-07-03-14-03-37
[2022-07-03 16:03:42] Uploaded result file: file-ptKx3VPOkc9fhDIFJrfR7PkX
[2022-07-03 16:03:43] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-14-03-37 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220703_160403-ft-pmx4lEQJw1L3rPR1PlHUzGXA
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-pmx4lEQJw1L3rPR1PlHUzGXA
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-pmx4lEQJw1L3rPR1PlHUzGXA
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                


🎉 wandb sync completed successfully


wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
wandb:                training_loss █▆▅▅▅▅▄▄▅▃█▄▃▃▄▃▄▃▄▂▂▃▃▂▃▃▃▃▄▁▂▂▁▁▂▃▂▂▂▂
wandb:   training_sequence_accuracy ▁█▁▁▁▁▁▁▁▁▁▁██▁█▁▁▁██▁█████▁▁█████▁▁████
wandb:      training_token_accuracy ▁█▁▁▁▁▁▁▁▁▁▁██▁█▁▁▁██▁█████▁▁█████▁▁████
wandb:              validation_loss █▂▂▂▄▂▂▂▂▂▂▂▂▂▂▂▄▂▁▂▁▂▂▁▂▁▃▂▂▂▁▂▁▂▂▁▁▂▁▁
wandb: validation_sequence_accuracy ▁▁▁█▁█▁▁▁█▁▁▁▁██▁▁▁▁█▁▁▁████████▁▁█▁████
wandb:    validation_token_accuracy ▁▆▆█▆█▆▆▆█▆▆▆▆██▆▆▆▆█▆▆▆████████▆▆█▆████
wandb: 
wandb: Run summary:
wandb:             elapsed_examples 2002.0
wandb:               elapsed_tokens 236098.0
wandb:             fine_tuned_model ada:ft-epfl-2022-07-...
wandb:                       status succeeded
wandb:                training_loss 0.02437
wandb:   training_sequence_accuracy 1.0
wandb:      training_token_accuracy 1.0
wandb:    

In [11]:
test_gpt3("ada:ft-epfl-2022-07-03-14-03-37", "ft-pmx4lEQJw1L3rPR1PlHUzGXA", valid_prompts, test_df)

{'ACC': {0: 0.91, 1: 0.85, 2: 0.82, 3: 0.83, 4: 0.93}, 'ACC_macro': 0.868, 'F1_micro': 0.67, 'F1_macro': 0.6703943460103294}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.868
F1_macro,0.67039
F1_micro,0.67


### GPT-3 on name

In [12]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=2)

Creating fine tune prompts for iupac_name


Upload progress: 100%|██████████| 65.3k/65.3k [00:00<00:00, 43.3Mit/s]


Uploaded file from run_files/2022-07-03-17-29-40_train_prompts_molecule_500.jsonl: file-KVj1DrSvO59plBgUnpKzGEur


Upload progress: 100%|██████████| 13.3k/13.3k [00:00<00:00, 22.7Mit/s]


Uploaded file from run_files/2022-07-03-17-29-40_validsmall_prompts_molecule_18009.jsonl: file-Ep30TnoK6PqKqgrbGN9s5SIV
Created fine-tune: ft-qaS5HnCXf2HALlhjhBclqtDC
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 17:29:48] Created fine-tune: ft-qaS5HnCXf2HALlhjhBclqtDC
[2022-07-03 17:29:57] Fine-tune costs $0.04
[2022-07-03 17:29:58] Fine-tune enqueued. Queue number: 0
[2022-07-03 17:30:04] Fine-tune started
[2022-07-03 17:31:30] Completed epoch 1/4
[2022-07-03 17:32:40] Completed epoch 2/4
[2022-07-03 17:33:49] Completed epoch 3/4
[2022-07-03 17:34:59] Completed epoch 4/4
[2022-07-03 17:35:25] Uploaded model: ada:ft-epfl-2022-07-03-15-35-23
[2022-07-03 17:35:28] Uploaded result file: file-gwW2lrlbfkPTuVEQ3G8gzh0O
[2022-07-03 17:35:28] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-15-35-23 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220703_173542-ft-qaS5HnCXf2HALlhjhBclqtDC
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-qaS5HnCXf2HALlhjhBclqtDC
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-qaS5HnCXf2HALlhjhBclqtDC
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                


🎉 wandb sync completed successfully


wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:                training_loss ▇▅▄▄▄▅▃▇▄█▅▂▂▂▄▄▃▄▃▅▂▂▃▃▂▂▃▂▁▃▂▅▁▃▂▂▂▃▁▃
wandb:   training_sequence_accuracy █████▁▁▁█▁▁████▁▁▁█▁█▁▁▁███▁█████████▁█▁
wandb:      training_token_accuracy █████▁▁▁█▁▁████▁▁▁█▁█▁▁▁███▁█████████▁█▁
wandb:              validation_loss █▂▁▂▁▁▂▁▂▂▁▁▁▁▂▂▂▁▂▂▁▁▂▂▁▁▁▁▁▂▂▁▁▃▁▁▂▃▂▂
wandb: validation_sequence_accuracy ▁▁█▁▁█▁▁▁▁█▁▁▁▁▁█▁▁▁███▁█████▁▁██▁██▁▁▁█
wandb:    validation_token_accuracy ▁▆█▆▆█▆▆▆▆█▆▆▆▆▆█▆▆▆███▆█████▆▆██▆██▆▆▆█
wandb: 
wandb: Run summary:
wandb:             elapsed_examples 2002.0
wandb:               elapsed_tokens 107578.0
wandb:             fine_tuned_model ada:ft-epfl-2022-07-...
wandb:                       status succeeded
wandb:                training_loss 0.08192
wandb:   training_sequence_accuracy 1.0
wandb:      training_token_accuracy 1.0
wandb:    

In [13]:
test_gpt3("ada:ft-epfl-2022-07-03-15-35-23", "ft-qaS5HnCXf2HALlhjhBclqtDC", valid_prompts, test_df, max_samples=200)

{'ACC': {0: 0.885, 1: 0.75, 2: 0.72, 3: 0.73, 4: 0.845}, 'ACC_macro': 0.7859999999999999, 'F1_micro': 0.465, 'F1_macro': 0.45763262770109636}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.786
F1_macro,0.45763
F1_micro,0.465


###  Large train set ($N=2000$)

In [7]:
train_df, test_df = train_test_split(df, train_size=2000, stratify=df[CAT_TARGETS[0]])
train_size = len(train_df)
test_size = len(test_df)
print(f"{len(train_df)} train points and {len(test_df)} test points")


2000 train points and 16509 test points


#### Train a baseline

In [7]:
run_baseline(train_df, test_df)


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/

{'ACC': {0: 0.9629898843055303, 1: 0.9213156460112666, 2: 0.9194984553879701, 3: 0.9308861832939609, 4: 0.9665636925313466}, 'ACC_macro': 0.9402507723060148, 'F1_micro': 0.8506269307650373, 'F1_macro': 0.8510644120566495}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.94025
F1_macro,0.85106
F1_micro,0.85063


### Fine tune on SMILES

In [8]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=0)


Creating fine tune prompts for smiles


Upload progress: 100%|██████████| 216k/216k [00:00<00:00, 120Mit/s]


Uploaded file from run_files/2022-07-03-19-53-03_train_prompts_molecule_2000.jsonl: file-dVryjqLtI4bTi2IzugGeOVA7


Upload progress: 100%|██████████| 11.2k/11.2k [00:00<00:00, 18.1Mit/s]


Uploaded file from run_files/2022-07-03-19-53-03_validsmall_prompts_molecule_16509.jsonl: file-jvcaIQJBgKmv3OJYq6T7EgGR
Created fine-tune: ft-asLUdFrceroByuDJmemXnUm6
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 19:53:15] Created fine-tune: ft-asLUdFrceroByuDJmemXnUm6
[2022-07-03 19:53:25] Fine-tune costs $0.16
[2022-07-03 19:53:26] Fine-tune enqueued. Queue number: 0
[2022-07-03 19:53:28] Fine-tune started
[2022-07-03 19:56:47] Completed epoch 1/4
[2022-07-03 19:59:49] Completed epoch 2/4
[2022-07-03 20:02:52] Completed epoch 3/4
[2022-07-03 20:05:55] Completed epoch 4/4
[2022-07-03 20:06:21] Uploaded model: ada:ft-epfl-2022-07-03-18-06-20
[2022-07-03 20:06:24] Uploaded result file: file-XcdLDGRcgJvD83aFMzFN6Wyy
[2022-07-03 20:06:25] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-18-06-20 -p <YOUR_PROMPT>


wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.12.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.12.19
wandb: Run data is saved locally in /Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/molecules/wandb/run-20220703_200645-ft-asLUdFrceroByuDJmemXnUm6
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-asLUdFrceroByuDJmemXnUm6
wandb: ⭐️ View project at https://wandb.ai/kjappelbaum/GPT-3
wandb: 🚀 View run at https://wandb.ai/kjappelbaum/GPT-3/runs/ft-asLUdFrceroByuDJmemXnUm6
wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                


🎉 wandb sync completed successfully


wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
wandb:                training_loss ▃▄▄▃▂▆▃▃▃▅▃▂▃█▃▃▃▃▃▅▃▁▂▃▃▃▃▃▄▂▁▃▄▃▃▂▂▂▂▃
wandb:   training_sequence_accuracy ▃▅▅▁▅▃▆▅▃▆▅▆▆▃▁█▃▆█▆▅▆█▆▃▆▅██▅██▆▆▃▆▅██▅
wandb:      training_token_accuracy ▃▄▄▁▄▃▆▄▃▆▄▆▆▃▁█▃▆█▆▄▆█▆▃▆▄██▄██▆▆▃▆▄██▄
wandb:              validation_loss █▃▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂
wandb: validation_sequence_accuracy ▁▃▅█▆█▅▅▆▃▅▅▃▅█▆▆▆▃▆█▅▆▅▆▅▆▆▆▅█▅▃▃▅▆▆▃▅█
wandb:    validation_token_accuracy ▁▆▇█▇█▇▇▇▆▇▇▆▇█▇▇▇▆▇█▇▇▇▇▇▇▇▇▇█▇▆▆▇▇▇▆▇█
wandb: 
wandb: Run summary:
wandb:             elapsed_examples 8008.0
wandb:               elapsed_tokens 755368.0
wandb:             fine_tuned_model ada:ft-epfl-2022-07-...
wandb:                       status succeeded
wandb:                training_loss 0.03892
wandb:   training_sequence_accuracy 1.0
wandb:      training_token_accuracy 1.0
wandb:    

In [9]:
test_gpt3("ada:ft-epfl-2022-07-03-18-06-20", "ft-asLUdFrceroByuDJmemXnUm6", valid_prompts, test_df, max_samples=200)

wandb: Currently logged in as: kjappelbaum. Use `wandb login --relogin` to force relogin


{'ACC': {0: 0.96, 1: 0.885, 2: 0.85, 3: 0.855, 4: 0.9}, 'ACC_macro': 0.89, 'F1_micro': 0.725, 'F1_macro': 0.719282006837113}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.89
F1_macro,0.71928
F1_micro,0.725


#### Fine-tune on SELFIEs

In [10]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=1)


Creating fine tune prompts for selfies


Upload progress: 100%|██████████| 499k/499k [00:00<00:00, 312Mit/s]


Uploaded file from run_files/2022-07-03-21-30-15_train_prompts_molecule_2000.jsonl: file-GeYUxJPeQVqH1lsjCP9qhPdH


Upload progress: 100%|██████████| 25.2k/25.2k [00:00<00:00, 45.0Mit/s]


Uploaded file from run_files/2022-07-03-21-30-15_validsmall_prompts_molecule_16509.jsonl: file-t1xe9Bh1wjYNjtAi8rxrkkK3
Created fine-tune: ft-6x5ZSdFqKldtSOJN8L4JlfQ4
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 21:30:24] Created fine-tune: ft-6x5ZSdFqKldtSOJN8L4JlfQ4
[2022-07-03 21:30:30] Fine-tune costs $0.38
[2022-07-03 21:30:30] Fine-tune enqueued. Queue number: 0
[2022-07-03 21:30:34] Fine-tune started
[2022-07-03 21:33:56] Completed epoch 1/4
[2022-07-03 21:37:02] Completed epoch 2/4
[2022-07-03 21:40:10] Completed epoch 3/4
[2022-07-03 21:43:14] Completed epoch 4/4
[2022-07-03 21:43:37] Uploaded model: ada:ft-epfl-2022-07-03-19-43-35
[2022-07-03 21:43:41] Uploaded result file: file-syLXukeauGZtQpbwKzFiCxdP
[2022-07-03 21:43:41] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-19-43-35 -p <YOUR_PROMPT>
Fin

In [11]:
test_gpt3("ada:ft-epfl-2022-07-03-19-43-35", "ft-6x5ZSdFqKldtSOJN8L4JlfQ4", valid_prompts, test_df, max_samples=200)

{'ACC': {0: 0.95, 1: 0.855, 2: 0.84, 3: 0.835, 4: 0.91}, 'ACC_macro': 0.8779999999999999, 'F1_micro': 0.695, 'F1_macro': 0.6902066086243617}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.878
F1_macro,0.69021
F1_micro,0.695


#### Fine-tune on chemical name

In [12]:
train_prompts, valid_prompts = create_prompt_fine_tune(train_df, test_df, text=2)


Creating fine tune prompts for iupac_name


Upload progress: 100%|██████████| 271k/271k [00:00<00:00, 105Mit/s]


Uploaded file from run_files/2022-07-03-22-30-58_train_prompts_molecule_2000.jsonl: file-YLYPRIXt6c4putPYfUW9y9UX


Upload progress: 100%|██████████| 13.3k/13.3k [00:00<00:00, 25.1Mit/s]


Uploaded file from run_files/2022-07-03-22-30-58_validsmall_prompts_molecule_16509.jsonl: file-dVTOz1B4ZrDROlXUlcIIBOF4
Created fine-tune: ft-DnPs5LPxYEX7ChafDaQgropw
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-03 22:31:07] Created fine-tune: ft-DnPs5LPxYEX7ChafDaQgropw
[2022-07-03 22:31:19] Fine-tune costs $0.17
[2022-07-03 22:31:19] Fine-tune enqueued. Queue number: 0
[2022-07-03 22:31:23] Fine-tune started
[2022-07-03 22:34:40] Completed epoch 1/4
[2022-07-03 22:37:41] Completed epoch 2/4
[2022-07-03 22:40:42] Completed epoch 3/4
[2022-07-03 22:43:44] Completed epoch 4/4
[2022-07-03 22:44:06] Uploaded model: ada:ft-epfl-2022-07-03-20-44-04
[2022-07-03 22:44:09] Uploaded result file: file-RPFWnN6tbnsd84gGvLV62Gg0
[2022-07-03 22:44:10] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-epfl-2022-07-03-20-44-04 -p <YOUR_PROMPT>
Fin

In [40]:
test_gpt3("ada:ft-epfl-2022-07-03-10-09-06", "ft-kTPe2DuwB9twH9wsPpBc2cDi", valid_prompts, test_df, max_samples=200)

{'ACC': {0: 0.935, 1: 0.86, 2: 0.825, 3: 0.86, 4: 0.93}, 'ACC_macro': 0.882, 'F1_micro': 0.705, 'F1_macro': 0.6999996459233866}


ACC_macro,▁
F1_macro,▁
F1_micro,▁
ACC_macro,0.882
F1_macro,0.7
F1_micro,0.705
